In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkConf, SparkContext
import numpy as np

In [2]:
conf = SparkConf().setMaster("local").setAppName("KMeans")
sc = SparkContext(conf = conf)

In [3]:
c1 = sc.textFile("hw3-q2-kmeans\\c1.txt")
c2 = sc.textFile("hw3-q2-kmeans\\c2.txt")
data = sc.textFile("hw3-q2-kmeans\\data.txt")

load the initial points and the data 

In [4]:
c1 = [x.split(" ") for x in c1.collect()]
c2 = [x.split(" ") for x in c2.collect()]

change the initial points into list for later usage

In [5]:
# pyspark how to pass parameters into mapper
# https://stackoverflow.com/questions/33019420/spark-rdd-mapping-with-extra-arguments
def Distance_process(c1,dist):
    def Euclidean_dist(x):
        D = []
        x = x.split(" ")
        for i in c1:
            dist = np.sqrt(sum([(float(a)-float(b))**2 for a,b in zip(i,x)]))
            D.append(dist)
        m = np.argmin(D)
        return (m,(x,min(D)**2))
    
    def Manhanttan_dist(x):
        D = []
        x = x.split(" ")
        for i in c1:
            dist = sum([abs(float(a)-float(b)) for a,b in zip(i,x)])
            D.append(dist)
        m = np.argmin(D)
        return (m,(x,min(D)))
    
    if dist == "Euclidean":
        return Euclidean_dist
    elif dist == "Manhanttan":
        return Manhanttan_dist
    else:
        raise 'must be either Euclidean or Manhanttan'

in order to pass down arguments into the mapping function, we can create an anonymous function
and pass arguments through it and inside we compute the Euclidean distance and Manhanttan distance and output the key-value pair
(closest_centroid, (the_point, the distance))

In [6]:
def New_points_mapper(x):
    for i in range(58):
        yield ((x[0],i),str(x[1][0][i]+" "))

in order to average of the points, we first create a key-value pair
(which_centroid, which_element_in_the_centroid, each_value_in_str)
later we will use the reduceByKey to augment them together
and with str value we can count how many points are belongs to this cluster

In [7]:
def New_pts(x):
    v_l = x[1].split(" ")[:-1]
    v = sum([float(x) for x in v_l])/len(v_l)
    return [x[0][0],x[0][1],v]

with the previous mapper function we can easily compute the average and output the pair of 
(which_centroid, which_element_in_the_centroid, new_element_of_the_points) 

In [8]:
def sum_cost(x):
    return ((1),x[1][1])

create a mapper to speed up the counting process of the total sum

In [9]:
def RUN_ALL(center_set, distance):
    COST = []
    times = 0 
    while times < 20:
        part_1 = data.map(Distance_process(center_set,distance))

        cost = part_1.map(sum_cost).reduceByKey(lambda p,q:p+q).collect()

        new_pt = part_1.flatMap(New_points_mapper).reduceByKey(lambda p,q:p+q).map(New_pts).collect()

        New_center = [new_pt[i:i+58] for i in range(0,580,58)]
        N = []
        for i in range(10):
            n = []
            for j in range(58):
                n.append(str(New_center[i][j][2]))
            N.append(n)
        center_set = N
        COST.append(cost)
        print(times) 
        times = times + 1
    
    return COST, center_set

here we augment all the mapper and reducer to get our wanted result, including the cost and the certroids

In [10]:
c1_Euc_COST,c1_Euc_Center = RUN_ALL(c1,"Euclidean")
c1_Man_COST,c1_Man_Center = RUN_ALL(c1,"Manhanttan")
c2_Euc_COST,c2_Euc_Center = RUN_ALL(c2,"Euclidean")
c2_Man_COST,c2_Man_Center = RUN_ALL(c2,"Manhanttan")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


we run the code with different initial points and different distance function

In [11]:
sc.stop()

In [12]:
with open("c1_Euc_Center.txt",'w') as f:
    for i in range(10):
        for j in range(58):
            f.write(str(c1_Euc_Center[i][j])+" ")
        f.write("\n")

with open("c1_Man_Center.txt",'w') as f:
    for i in range(10):
        for j in range(58):
            f.write(str(c1_Man_Center[i][j])+" ")
        f.write("\n")
        
with open("c2_Euc_Center.txt",'w') as f:
    for i in range(10):
        for j in range(58):
            f.write(str(c2_Euc_Center[i][j])+" ")
        f.write("\n")
    
with open("c2_Man_Center.txt",'w') as f:
    for i in range(10):
        for j in range(58):
            f.write(str(c2_Man_Center[i][j])+" ")
        f.write("\n")


we output each result to txt file 

In [18]:
with open("c1_Euc_COST.txt",'w') as f:
    for i in range(len(c1_Euc_COST)):
        f.write(str(c1_Euc_COST[i][0][1])+"\n")
        
with open("c1_Man_COST.txt",'w') as f:
    for i in range(len(c1_Man_COST)):
        f.write(str(c1_Man_COST[i][0][1])+"\n")
        
with open("c2_Euc_COST.txt",'w') as f:
    for i in range(len(c2_Euc_COST)):
        f.write(str(c2_Euc_COST[i][0][1])+"\n")
        
with open("c2_Man_COST.txt",'w') as f:
    for i in range(len(c2_Man_COST)):
        f.write(str(c2_Man_COST[i][0][1])+"\n")

In [52]:
def compute_distance_btw_centroid(x):
    EUC = []
    MAN = []
    for i in range(10):
        for j in range(i,10):
            euc = 0
            man = 0
            for k in range(len(x[0])):
                euc = euc + (float(x[i][k]) - float(x[j][k]))**2
                man = man + abs(float(x[i][k]) - float(x[j][k]))
            EUC.append(euc)
            MAN.append(man)
    return EUC, MAN

In [53]:
# c1_Euc_Center
EUC_c1_EUC, MAN_c1_EUC = compute_distance_btw_centroid(c1_Euc_Center)
with open("HW3_result\\centroid_distance\\EUC_c1_EUC.txt",'w') as f:
    for i in EUC_c1_EUC:
        f.write(str(i))
        f.write("\n")
        
with open("HW3_result\\centroid_distance\\MAN_c1_EUC.txt",'w') as f:
    for i in MAN_c1_EUC:
        f.write(str(i))
        f.write("\n")

In [54]:
# c1_Man_Center
EUC_c1_Man, MAN_c1_Man = compute_distance_btw_centroid(c1_Man_Center)
with open("HW3_result\\centroid_distance\\EUC_c1_Man.txt",'w') as f:
    for i in EUC_c1_Man:
        f.write(str(i))
        f.write("\n")
        
with open("HW3_result\\centroid_distance\\MAN_c1_Man.txt",'w') as f:
    for i in MAN_c1_Man:
        f.write(str(i))
        f.write("\n")

In [55]:
# c2_Euc_Center
EUC_c2_EUC, MAN_c2_EUC = compute_distance_btw_centroid(c2_Euc_Center)
with open("HW3_result\\centroid_distance\\EUC_c2_EUC.txt",'w') as f:
    for i in EUC_c2_EUC:
        f.write(str(i))
        f.write("\n")
        
with open("HW3_result\\centroid_distance\\MAN_c2_EUC.txt",'w') as f:
    for i in MAN_c2_EUC:
        f.write(str(i))
        f.write("\n")

In [56]:
# c2_Man_Center
EUC_c2_Man, MAN_c2_Man = compute_distance_btw_centroid(c2_Man_Center)
with open("HW3_result\\centroid_distance\\EUC_c2_Man.txt",'w') as f:
    for i in EUC_c2_Man:
        f.write(str(i))
        f.write("\n")
        
with open("HW3_result\\centroid_distance\\MAN_c2_Man.txt",'w') as f:
    for i in MAN_c2_Man:
        f.write(str(i))
        f.write("\n")